# Feature Generation pt.2

This module will determine the characteristics for each executive that we found in pt.1.  

## Features

## Data Sources

## Data Structure

Pandas DF with grouped index:

                      Feature 1    Feature 2
    Name    DataSource1  
            DataSource2  
            DataSource3  

In [18]:
# to work with data
import pandas as pd

# to work with regex
import re

# to download
import requests

# to work with HTML tags
from bs4 import BeautifulSoup

# to time functions
import datetime

# to use NaN
import numpy as np

# to pause
import time
import random

# to work with local files
import os

In [19]:
# restore the dataframe of CEO and CFO by quarter by company
'''
try:
    Company_Exec
except NameError:
    Company_Exec = pd.read_csv('Company_CEO_CFO.csv', index_col=0)
'''

"\ntry:\n    Company_Exec\nexcept NameError:\n    Company_Exec = pd.read_csv('Company_CEO_CFO.csv', index_col=0)\n"

In [20]:
#grouped = Company_Exec.groupby('cik')

In [21]:
#grouped.filter(lambda x: x['cik'].count() > 1)

In [22]:
#DataFrames['SUB'][DataFrames['SUB']['name'] == 'ADVANCED MICRO DEVICES INC']

In [23]:
Bio = """Dr. Lisa T. Su has been the Chief Executive Officer 
and President of Advanced Micro Devices, Inc. since August 10, 2014 and 
October 2014 respectively. Dr. Su served as Chief Operating Officer of 
Advanced Micro Devices, Inc. from July 1, 2014 to October 2014. She 
served as Senior Vice President and General Manager of Global Business 
Unit at Advanced Micro Devices, Inc. from January 3, 2012 to July 2014. 
She served as Senior Vice President of Freescale Semiconductor 
Holdings I Ltd. from June 18, 2007 to December 2011 and served as its 
General Manager of Networking and Multimedia from September 2008 to 
December 2011. She is a highly-respected technology executive and has 
an Outstanding Record of business execution. She has an impressive 
track record and solid understanding of the semiconductor industry. 
She served as Senior Vice President and General Manager of Networking 
and Multimedia of Freescale Semiconductor Inc. from 2007 to 2011. 
Dr. Su served as the Chief Technology Officer of Freescale 
Semiconductor Holdings I Ltd. from June 18, 2007 to August 2009. She 
served as the Chief Technology Officer of Freescale Semiconductor Inc. 
since June 18, 2007. Dr. Su is a respected industry leader with more 
than 13 years of experience in technology development, product 
management and strategic alliances. She served as Vice President of 
Semiconductor Research and Development Center at International Business 
Machines Corp. since October 2005. She joined Freescale from IBM, where 
she served in various capacities including Vice President of Semiconductor 
Research and Development, Vice President of Technology Development and 
Alliances in the IBM Systems and Technology Group. Dr. Su served as a 
Director of the PowerPCTMproduct line and Emerging Products from 1995 
to June 2007. She served in various technical and business positions at 
IBM including Vice President of Technology Development. She has been a 
Director of Analog Devices, Inc. since June 7, 2012 and Advanced Micro 
Devices, Inc. since October 8, 2014. She has authored or co-authored 
more than 40 technical publications and co-authored a book chapter on 
next-generation consumer electronics. She was named in MIT Technology 
Review's Top 100 Young Innovators in 2002 and received the YWCA 2003 
Outstanding Achievement Award for Business. Prior to IBM, she was a 
member of the technical staff at Texas Instruments Incorporated in the 
Semiconductor Process and Device Center from 1994 to 1995. She is a 
Fellow of the Institute of Electronics and Electrical Engineers (IEEE). 
Dr. Su received B.S., M.S., and Doctorate Degrees in Electrical 
Engineering from the Massachusetts Institute of Technology. She has 
published more than 40 technical articles and was named a Fellow of 
the Institute of Electronics and Electrical Engineers in 2009. Dr. Su 
was named “2014 Executive of the Year” at the EETimes and EDN 2014 ACE 
Awards and was honored in MIT Technology Review’s Top 100 Young 
Innovators in 2002.
"""

In [24]:
# First stab at feature extracting using rule-based logic
fields = ['Gender', 'Degree', 'Awards', 'Career Start', 'Ivy League']

df = pd.DataFrame()
df['name'] = ['Lisa T. Su']
df['Bio'] = Bio

def features_from_bio(Bio):
    # Determine gender based on pronouns in text
    gender = np.nan
    if 'she' in Bio:
        gender = 'Female'
    elif 'he' in Bio:
        gender = 'Male'
    
    
    # Determine degree based on mentions in text     
    degree = np.nan
    if 'B.S.' in Bio:
        degree = 'Bachelor'
    if 'M.S.' in Bio:
        degree = 'Master'    
    if 'Dr.' in Bio:
        degree = 'Doctorate'

    
    # Is there any mention of awards?
    awards = False
    if 'Award' or 'award' in Bio:
        awards = True

    
    # Assume the earliest year mentioned is the start of their career.    
    years = [int(x) for x in re.findall(r'\d{4}', str(Bio))]
    career_start = min(years)


    # Is there any mention of an Ivy League university?
    IvyLeague = ['Brown University', 'Columbia University', 'Cornell University', 'Dartmouth College', 'Harvard University', 'University of Pennsylvania', 'Princeton University', 'Yale University']
    ivy_league = False
    for uni in IvyLeague:
        if uni in Bio:
            ivy_league = True

    return gender, degree, awards, career_start, ivy_league
    
df['Gender'], df['Degree'], df['Awards'], df['Career Start'], df['Ivy League'] = \
zip(*df['Bio'].map(features_from_bio))

df.head()

,name,Bio,Gender,Degree,Awards,Career Start,Ivy League
0,Lisa T. Su,Dr. Lisa T. Su has been the Chief Executive Of...,Female,Doctorate,True,1994,False


### Information Extraction: Read this 
https://web.stanford.edu/~jurafsky/slp3/17.pdf

Paragraph 5 and 6 !!
https://www.nltk.org/book/ch07.html

### Further ideas - extracurriculars:
1. Contribute to WikiData with this information.
2. Write blog post about information extraction with Python (Very little information available).
